In [131]:
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

fdir = '/Users/naveenkumar/Dropbox/OnlineWork/Datasets/ElevatorProblem/'

ds = pd.read_csv(fdir + 'input.csv')
ds

,Person_Id,Floor_in,Floor_in_time,Elevator_in_time,Elevator_out_time,Floor_out
0,1,0,03-04-2018 23:10:15,03-04-2018 23:12:30,03-04-2018 23:13:39,7
1,1,2,03-04-2018 11:10:15,03-04-2018 11:12:30,03-04-2018 11:13:39,6
2,1,4,05-04-2018 11:10:15,05-04-2018 11:12:30,05-04-2018 11:13:39,20
3,1,0,03-04-2018 11:10:15,03-04-2018 11:12:30,03-04-2018 11:13:39,7
4,2,0,03-04-2018 23:10:15,03-04-2018 23:12:30,03-04-2018 23:13:39,7
5,2,4,04-04-2018 23:10:15,04-04-2018 23:12:30,04-04-2018 23:13:39,6
6,2,3,03-04-2018 23:10:15,03-04-2018 23:12:30,03-04-2018 23:13:39,20
7,2,8,03-04-2018 23:10:15,03-04-2018 23:12:30,03-04-2018 23:13:39,7


In [127]:
def houroftheday(x):
    tmp = datetime.strptime(x, '%d-%m-%Y %H:%M:%S')
    return str(tmp.hour)

def getdate(x):
    tmp = datetime.strptime(x, '%d-%m-%Y %H:%M:%S')
    return str(tmp.date())

'2018-04-03'

Total unique floor accessed

In [137]:
Floor_in_Access = ds[['Person_Id', 'Floor_in', 'Elevator_in_time']]
Floor_out_Access = ds[['Person_Id', 'Floor_out', 'Elevator_in_time']]

Floor_in_Access.rename(columns={'Floor_in':'Floor_id'}, inplace=True)
Floor_out_Access.rename(columns={'Floor_out':'Floor_id'}, inplace=True)

All_Floor_Accessed = pd.concat([Floor_in_Access, Floor_out_Access])
All_Floor_Accessed['DayAccessed'] =  All_Floor_Accessed['Elevator_in_time'].apply(lambda col:getdate(col))
print All_Floor_Accessed.shape

Unique_Floor_Accessed = All_Floor_Accessed.drop_duplicates(['Person_Id', 'Floor_id','DayAccessed'])
print Unique_Floor_Accessed.shape
#Unique_floor_accessed_count = Unique_Floor_Accessed[['Person_Id', 'Floor_id']].groupby(['Person_Id'], as_index=False).count()
#Unique_floor_accessed_count.rename(columns={'Floor_id':'NumUniqueFloorsAccessed'}, inplace=True)

(16, 4)
(13, 4)


#Total Journeys in odd hours/ Total Journeys


Hour Oddness Score

In [115]:
Ele_Access_Hours = ds[['Person_Id', 'Elevator_in_time']]
Ele_Access_Hours['Ele_Access_Hour'] = Ele_Access_Hours['Elevator_in_time'].apply(lambda col:houroftheday(col))
Ele_Access_Hours

,Person_Id,Elevator_in_time,Ele_Access_Hour
0,1,03-04-2018 23:12:30,23
1,1,03-04-2018 11:12:30,11
2,1,03-04-2018 11:12:30,11
3,1,03-04-2018 11:12:30,11
4,2,03-04-2018 23:12:30,23
5,2,03-04-2018 23:12:30,23
6,2,03-04-2018 23:12:30,23
7,2,03-04-2018 23:12:30,23


In [116]:
Ele_Access_Hours_count = Ele_Access_Hours[['Person_Id', 'Ele_Access_Hour']].groupby(['Ele_Access_Hour'], as_index=False).count()
Ele_Access_Hours_count.rename(columns={'Person_Id':'NumTimeFloorAccessed'}, inplace=True)
Ele_Access_Hours_count['HourOddnessScore'] = np.power(Ele_Access_Hours_count['NumTimeFloorAccessed'].rank(ascending = False),1.5)




In [122]:
Ele_Access_Hours_Oddness = pd.merge(Ele_Access_Hours, Ele_Access_Hours_count[['Ele_Access_Hour', 'HourOddnessScore']], on='Ele_Access_Hour', how='left')
print Ele_Access_Hours_Oddness
HouseOddnessScore = Ele_Access_Hours_Oddness[['Person_Id', 'HourOddnessScore']].groupby(['Person_Id'], as_index=False).mean()
print HouseOddnessScore

   Person_Id     Elevator_in_time Ele_Access_Hour  HourOddnessScore
0          1  03-04-2018 23:12:30              23          1.000000
1          1  03-04-2018 11:12:30              11          2.828427
2          1  03-04-2018 11:12:30              11          2.828427
3          1  03-04-2018 11:12:30              11          2.828427
4          2  03-04-2018 23:12:30              23          1.000000
5          2  03-04-2018 23:12:30              23          1.000000
6          2  03-04-2018 23:12:30              23          1.000000
7          2  03-04-2018 23:12:30              23          1.000000
   Person_Id  HourOddnessScore
0          1           2.37132
1          2           1.00000


In [94]:
Ele_Access_Hours[['Person_Id', 'Ele_Access_Hour']].groupby(['Person_Id','Ele_Access_Hour']).count()

Empty DataFrame
Columns: []
Index: [(1, 11), (1, 23), (2, 23)]